In [1]:
# Imports 
import time
import os
import re
import json
import collections

import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

# Load your usual SpaCy model (one of SpaCy English models)
import spacy
nlp = spacy.load('en')

# Add neural coref to SpaCy's pipe
import neuralcoref
neuralcoref.add_to_pipe(nlp, blacklist=False)

[nltk_data] Downloading package stopwords to /home/dmac/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# import torch
# from transformers import AutoTokenizer, AutoModelForTokenClassification
# from transformers import pipeline 

# tokenizer = AutoTokenizer.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")

# model = AutoModelForTokenClassification.from_pretrained("vblagoje/bert-english-uncased-finetuned-pos")

# nlp_token_class = pipeline('ner', model=model, tokenizer=tokenizer, grouped_entities=True)

In [81]:
'''
A class reresenting a mention of some entity. 
It consists of a text string, starting and ending indices
of the text span and a paragraph and sentence id. 
'''
class Mention:
    def __init__(self, text, start, end, par_id, sent_id):
        self.text =  text
        self.start = start
        self.end = end
        self.par_id = par_id
        self.sent_id = sent_id
        self.POS = []
        self.deps = set()
        
    def __repr__(self):
        rep = "Mention in par " + str(self.par_id) + " Sentence " \
              + str(self.sent_id) + " text:\n" + self.text
        return rep

'''
A class reresenting a character. 
It consists of a list of mentions, a set of aliases, 
a list of verbs that the character is the actor of (agent),
a list of verbs that the character is the receiver of (patient),
a list of adjectives that the character is described with (description).  
'''
class Character:
    def __init__(self, book, mainName):
        self.book = book
        self.mainName = mainName
        self.mentions = []
        self.unique_sents = {}
        self.aliases = set()
        self.agent = {}
        self.patient = []
        self.description = []
        
    def __repr__(self):
        rep = "Character: " + self.mainName + "\n"
        return rep
    
    '''
    Add a mention of the character to a list of mentions.
    '''
    def update_mention(self, mention):
        self.mentions.append(mention)
        self.aliases.update([mention.text.lower()])
        
    '''
    Match POS tags with character mentions
    '''
    def get_POS(self):
        for mention in self.mentions:
            span = range(mention.start, mention.end)
            for loc in span:
                mention.POS.append(self.book.pars[mention.par_id].POS_tags[loc])
                mention.deps.update([mention.POS[-1]['dep']])
                
    def get_unique_sent_mentions(self):
        for mention in self.mentions:
            self.unique_sents[(mention.par_id, mention.sent_id)] = \
            self.book.pars[mention.par_id].sents[mention.sent_id].text
                
    '''
    Function to find the verbs in sentences in which the character is mentioned 
    as the nsubj.
    '''
    def get_agent_verbs(self):
        for mention in self.mentions:
            verb = None
            if 'nsubj' in mention.deps:
                sent_POS_parse = self.book.pars[mention.par_id].sents[mention.sent_id].POS_tags
                for POS in sent_POS_parse:
                    if POS['dep'] == 'ROOT':
                        verb = POS['text']
                        # Get location tuple (paragraph, sentence, local idx)
                        global_loc = (mention.par_id, mention.sent_id, POS['loc'])
                if verb:
                    self.agent[global_loc] = verb
                    
        

class Sentence:
    def __init__(self, book, par, text, sent_id, bound):
        # Parent objects
        self.book = book
        self.par = par
        # starting and ending boundary for the sentence (relative to the paragraph)
        self.start = bound[0]
        self.end = bound[1]
        # Global starting and ending indices
        self.globalStart = self.par.start + self.start
        self.globalEnd = self.par.start + self.end 
        self.text = text
        self.sent_id = sent_id
        self.POS_tags = []
        
    def __repr__(self):
        rep = self.book.fileName + "\nParagraph " + str(self.par.par_id) \
        + "\nSentence " + str(self.sent_id) + " start " + str(self.start) \
        + " end " + str(self.end) + "\ntext:\n" + self.text
        return rep
        
'''
A class for a paragraph.
'''
class Paragraph:
    def __init__(self, book, text, par_id, bound):
        # Starting and ending indices for the paragraph
        self.start = bound[0]
        self.end = bound[1]
        # Refers to book parent object
        self.book = book
        # paragraph text
        self.text = text
        # paragraph index
        self.par_id = par_id
        # bool idnicating whether coref wroked
        self.has_coref = False
        # coref cluster list
        self.coref_clusts = []
        # list of sentences
        self.sents = []
        self.sent_bounds = []
        self.POS_tags = []
        
    def __repr__(self):
        rep = self.book.fileName + "\nParagraph " + str(self.par_id) + "\ntext:\n" + self.text
        return rep
    
    '''
    Split text into sentences
    '''
    def parse_into_sentences(self):
        doc = nlp(self.text)
        sentences = [sent for sent in doc.sents]
        self.sents = [Sentence(self.book, self, sent.text, 
                               sent_id, (sent.start, sent.end)) 
                      for sent_id, sent in enumerate(sentences)]
        
    def run_coref_POS(self):
        # Get coreference tags:
        doc = nlp(self.text)
        # Update whether the coref parser returned results
        self.has_coref = doc._.has_coref
        if doc._.has_coref:
            self.coref_clusts = doc._.coref_clusters
            self.book.parse_coref_clusts(self)
        # Get POS tags:

        # this uses the pretrained BERT model:
        # book[idx]["POS_tags"] = nlp_token_class(sentence)

        # For now we will use Spacy pos tagging because it gives us more fine-grained labels:
        self.POS_tags = []
        # Add POS tags to list for paragraph and for each individual sentence
        for token_id in range(len(doc)):
            token = doc[token_id]
            sent_id = 0
            for sent in self.sents:
                if ((token_id >= sent.start) and (token_id <= sent.end)):
                        sent_id = sent.sent_id
            self.sents[sent_id].POS_tags.append({
                'loc' : token_id,
                'text' : token.text,
                'lemma' : token.lemma_,
                'pos' : token.pos_, 
                'tag' : token.tag_, 
                'dep' : token.dep_})
            self.POS_tags.append({
                'loc' : token_id,
                'text' : token.text,
                'lemma' : token.lemma_,
                'pos' : token.pos_, 
                'tag' : token.tag_, 
                'dep' : token.dep_})

'''
A class representing a book.
'''        
class Book:
    def __init__(self, dataPath, fileName):
        self.dataPath = dataPath
        self.fileName = fileName
        self.text = ''
        self.characters = {}
        self.pars = []
        self.read_file()
    
    '''
    Read the text of the book from a txt file.
    '''
    def read_file(self):
        with open(os.path.join(self.dataPath, self.fileName), "r") as txtFile:
            self.text = txtFile.read()
        
    '''
    Break the text into paragraphs.
    '''
    def parse_into_pars(self):
        # split on newlines followed by space
        pars = re.split('\n\s', self.text)   
        par_bounds = [0]
        par_bounds += [m.start(0) for m in re.finditer('\n\s', self.text)]
        par_bounds.append(len(self.text) - 1)
        # Replace newline chars
        pars = [par.replace("\n", " ") for par in pars]
        # Remove empty pars
        pars = [par for par in pars if len(par) > 0]
        # Convert each paragraph into a Paragraph
        self.pars = [Paragraph(self, par, par_id, (par_bounds[par_id],
                                                   par_bounds[par_id+1])) 
                     for par_id, par in enumerate(pars)]
        
    '''
    Parse the coreference clusters returned from the parsing of a paragraph
    '''
    def parse_coref_clusts(self, par):
        clustList = par.coref_clusts
        # Iterate over the coreference clusters
        for idx, cluster in enumerate(clustList):
            # get the main cluster identity
            mainSpan = cluster.main.text.lower()
            # If a character object does not yet exist, create one
            if mainSpan not in self.characters:
                character = Character(self, mainSpan)
                # Add it to the dict of characters
                self.characters[mainSpan] = character
            # Otherwsie find the character referred to here
            else:
                character = self.characters[mainSpan] 
            for mention in cluster.mentions:
                # figure out which sentence the mention belongs to
                sent_id = 0
                for sent in par.sents:
                    if ((mention.start >= sent.start) and (mention.end <= sent.end)):
                        sent_id = sent.sent_id
                # create a mention object and add it to the character object
                mention = Mention(mention.text, mention.start, mention.end, par.par_id, sent_id)
                character.update_mention(mention)
                
    def parse_text(self):
        self.parse_into_pars()
        for par in self.pars:
            par.parse_into_sentences()
            par.run_coref_POS()
        for characterName, character in self.characters.items():
            character.get_POS()
            character.get_agent_verbs()
            character.get_unique_sent_mentions()

In [82]:
# Define Paths and fileNames:
dataPath = "../../Data/"
fileName = "Herman Melville___Bartleby, The Scrivener.txt"
stopwordsFileName = "StopWords/jockers.stopwords"
outFileName = 'Herman Melville___Bartleby, The Scrivener_sentences.json'

In [83]:
b = Book(dataPath, fileName)
b.parse_text()

In [ ]:
for character in b.characters:
    for loc, sent in character.unique_sents.items():
        emote = get_emotion(sent)
        print(loc, emote)

In [100]:
b.characters["it"].get_unique_sent_mentions()
b.characters["it"].unique_sents

{(4,
  6): 'I do not speak it in vanity, but simply record the fact, that I was not unemployed in my profession by the late John Jacob Astor; a name which, I admit, I love to repeat, for it hath a rounded and orbicular sound to it, and rings like unto bullion.  ',
 (14,
  0): 'It was fortunate for me that, owing to its peculiar cause--indigestion--the irritability and consequent nervousness of Nippers, were mainly observable in the morning, while in the afternoon he was comparatively mild.  ',
 (84, 0): 'Shall I acknowledge it?  ',
 (84,
  1): 'The conclusion of this whole business was, that it soon became a fixed fact of my chambers, that a pale young scrivener, by the name of Bartleby, and a desk there; that he copied for me at the usual rate of four cents a folio (one hundred words); but he was permanently exempt from examining the work done by him, that duty being transferred to Turkey and Nippers, one of compliment doubtless to their superior acuteness; moreover, said Bartleby was

In [84]:
b.characters["bartleby"].get_unique_sent_mentions()
b.characters["bartleby"].unique_sents

{(2,
  4): 'While of other law-copyists I might write the complete life, of Bartleby nothing of that sort can be done.',
 (2,
  7): 'Bartleby was one of those beings of whom nothing is ascertainable, except from the original sources, and in his case those are very small.  ',
 (2,
  8): 'What my own astonished eyes saw of Bartleby, _that_ is all I know of him, except, indeed, one vague report which will appear in the sequel.',
 (18,
  2): 'I resolved to assign Bartleby a corner by the folding-doors, but on my side of them, so as to have this quiet man within easy call,',
 (18,
  6): 'Still further to a satisfactory arrangement, I procured a high green folding screen, which might entirely isolate Bartleby from my sight, though not remove him from my voice.  ',
 (21,
  3): 'In my haste and natural expectancy of instant compliance, I sat with my head bent over the original on my desk, and my right hand sideways, and somewhat nervously extended with the copy, so that immediately upon emergi

In [85]:
for loc, sent in b.characters["bartleby"].unique_sents.items():
    #print(sent)
    emote = get_emotion(sent)
    print(loc, emote)

(2, 4) anger
(2, 7) fear
(2, 8) surprise
(18, 2) joy
(18, 6) joy
(21, 3) fear
(27, 0) anger
(27, 1) fear
(27, 2) joy
(27, 4) joy
(51, 0) sadness
(53, 5) love
(53, 7) anger
(69, 0) anger
(69, 3) joy
(85, 0) joy
(85, 6) fear
(85, 7) joy
(88, 4) sadness
(88, 7) anger
(88, 10) joy
(88, 12) anger
(89, 3) fear
(89, 4) fear
(89, 5) fear
(89, 8) joy
(89, 10) sadness
(89, 14) joy
(110, 2) joy
(110, 4) fear
(121, 0) anger
(127, 0) anger
(127, 1) anger
(134, 1) joy
(134, 3) fear
(134, 4) anger
(134, 5) anger
(137, 11) anger
(137, 12) fear
(137, 13) joy
(137, 14) anger
(137, 15) anger
(146, 3) sadness
(146, 8) sadness
(148, 4) anger
(148, 6) joy
(149, 1) sadness
(156, 7) sadness
(156, 8) anger
(156, 10) fear
(156, 12) anger
(166, 0) anger
(167, 2) joy
(168, 2) fear
(168, 3) joy
(170, 0) anger
(170, 1) sadness
(170, 6) fear
(171, 0) anger
(171, 1) joy
(171, 2) joy
(222, 0) sadness


In [78]:
b.characters["bartleby"].agent

{(2, 7, 183): 'was',
 (21, 3, 123): 'sat',
 (27, 0, 7): 'concluded',
 (27, 1, 31): 'became',
 (27, 2, 39): 'was',
 (51, 0, 2): 'sat',
 (53, 5, 107): 'Was',
 (69, 3, 33): 'remembered',
 (88, 4, 141): 'was',
 (88, 7, 192): 'was',
 (88, 10, 228): 'was',
 (89, 3, 39): 'was',
 (89, 5, 75): 'surmised',
 (89, 8, 185): 'thought',
 (89, 14, 301): 'makes',
 (110, 2, 86): 'sat',
 (121, 0, 11): 'said',
 (127, 0, 4): 'noticed',
 (127, 1, 32): 'said',
 (134, 3, 41): 'vouchsafed',
 (134, 4, 52): 'do',
 (134, 5, 67): 'informed',
 (137, 11, 182): 'told',
 (137, 14, 230): 'take',
 (148, 4, 67): 'was',
 (156, 7, 163): 'assumed',
 (156, 8, 192): 'enter',
 (156, 10, 239): 'was',
 (167, 2, 51): 'emerge',
 (170, 0, 36): 'request',
 (170, 1, 63): 'decline',
 (170, 6, 143): 'came',
 (171, 2, 57): 'apprised'}

In [ ]:
bart_sent_mentions

#### Below is some first pass testing that is commented out.

In [ ]:
# '''
# Split text into sentences
# '''
# def parse_into_sentences(fileName):
#     with open(os.path.join(dataPath, fileName), "r") as txtFile:
#         doc = txtFile.read()
#     sentences = re.split('\.|!|\?', doc)   
#     sentences = [sentence.replace("\n", " ") for sentence in sentences]
#     return sentences
# '''
# Split text into paragraphs
# '''
# def parse_into_pars(fileName):
#     with open(os.path.join(dataPath, fileName), "r") as txtFile:
#         doc = txtFile.read()
#     pars = re.split('\n\s', doc)   
#     pars = [par.replace("\n", " ") for par in pars]
#     return pars
# '''
# Create list of stop words (currently not used)
# '''
# def read_stopwords(filename):
#     stopwords={}
#     with open(filename) as file:
#         for line in file:
#             stopwords[line.rstrip()]=1
#     return stopwords

# '''
# Clean sentences by removing trailing punctuation on words, 
# and converting to lowercase
# '''
# def clean_sentences(sentences, charsTOStrip = '\"\', '):
#     texts = [
#         [word.strip(charsTOStrip) for word in sentence.lower().split()]
#         for sentence in sentences]
#     return texts

In [ ]:
# '''
# Convert list of neuralcoref.neuralcoref.Cluster objects 
# to list of dicts so that we can serialize it. 
# '''
# def convert_clust_to_list_dict(clustList):
#     dictList = []
#     characters = {character : {
#     "aliases" : set(),
#     "agent" : [],
#     "patient" : [],
#     "description" : [],
#     } for character in top_characters}
    
#     for idx, cluster in enumerate(clustList):
#         mainSpan = cluster.main
#         dictList.append({mainSpan.text.lower() : [{'start' : mention.start, 'end': mention.end, 'text' : mention.text}
#                                            for mention in cluster.mentions]})
#         characters[mainSpan.text.lower()] += 1
#     return dictList, characters

In [ ]:
# # Creating book data structure that we will use.
# # For now it is just a dict, but maybe we can make it a class
# def create_book_struct(fileName):
    
#     characters = collections.Counter()
    
#     time_start = time.time()

#     # break file into paragraphs
#     pars = parse_into_pars(fileName)
#     #parse_into_sentences(fileName)

#     # initialize book obj
#     book = {}
#     # Iterate over each sentence and 
#     for idx, sentence in enumerate(sentences):

#         # Get sentence id and text
#         book[idx] = {}
#         book[idx]['text'] = sentence

#         # Get coreference tags:
#         doc = nlp(sentence)
#         book[idx]['has_coref'] = doc._.has_coref
#         if doc._.has_coref:
#             coref_clusts, chars = convert_clust_to_list_dict(doc._.coref_clusters)
#             book[idx]['coref_clusts'] = coref_clusts
            
#             characters += chars
#         else:
#             book[idx]['coref_clusts'] = []

#         # Get POS tags:

#         # this uses the pretrained BERT model:
#         # book[idx]["POS_tags"] = nlp_token_class(sentence)

#         # For now we will use Spacy pos tagging because it gives us more fine-grained labels:
#         book[idx]["POS_tags"] = []
#         for token in doc:
#             book[idx]["POS_tags"].append({
#                 'text' : token.text,
#                 'lemma' : token.lemma_,
#                 'pos' : token.pos_, 
#                 'tag' : token.tag_, 
#                 'dep' : token.dep_})
#     time_end = time.time()
#     print("Parsing book took ", round(time_end - time_start, 2), " secs")
#     return book, characters

In [ ]:
# book, characters = create_book_struct(fileName)

In [ ]:
# def write_book_to_disk(book, outFileName):
#     with open (os.path.join(dataPath, outFileName), 'w') as outFile:
#         json.dump(book, outFile, separators=(',', ':'), indent=4)

In [ ]:
# write_book_to_disk(book, outFileName)

In [ ]:
# # Only get top n most mentioned characters. 
# # Probably we should replace this with some kind of 
# # filter based on the min number of mentions
# most_common_n = 20
# top_characters = characters.most_common()[:most_common_n]
# top_characters = [top_character[0] for top_character in top_characters]

In [ ]:
# doc = nlp('''
# I am a rather elderly man.  The nature of my avocations for the last
# thirty years has brought me into more than ordinary contact with what
# would seem an interesting and somewhat singular set of men, of whom as
# yet nothing that I know of has ever been written:--I mean the
# law-copyists or scriveners.  I have known very many of them,
# professionally and privately, and if I pleased, could relate divers
# histories, at which good-natured gentlemen might smile, and sentimental
# souls might weep.  But I waive the biographies of all other scriveners
# for a few passages in the life of Bartleby, who was a scrivener of the
# strangest I ever saw or heard of.  While of other law-copyists I might
# write the complete life, of Bartleby nothing of that sort can be done.
# I believe that no materials exist for a full and satisfactory biography
# of this man.  It is an irreparable loss to literature.  Bartleby was one
# of those beings of whom nothing is ascertainable, except from the
# original sources, and in his case those are very small.  What my own
# astonished eyes saw of Bartleby, _that_ is all I know of him, except,
# indeed, one vague report which will appear in the sequel.
# ''')

# for token in doc:
#     print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
#             token.shape_, token.is_alpha)

In [19]:
from transformers import AutoTokenizer, AutoModelWithLMHead

tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-emotion")

model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-emotion")


/home/dmac/Documents/MIMS Coursework/Info 190 Comp Hum/Project/code/.env/lib/python3.8/site-packages/transformers/modeling_auto.py:781: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


/home/dmac/Documents/MIMS Coursework/Info 190 Comp Hum/Project/code/.env/lib/python3.8/site-packages/transformers/tokenization_t5.py:184: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(


'sadness'

In [93]:
def get_emotion(text):
    input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')

    output = model.generate(input_ids=input_ids,
               max_length=2)

    dec = [tokenizer.decode(ids) for ids in output]
    label = dec[0]
    return output

get_emotion("i feel as if i havent blogged in ages are at least truly blogged i am doing an update cute") # Output: 'joy'

#get_emotion("i have a feeling i kinda lost my best friend") # Output: 'sadness'

tensor([[   0, 3922]])

In [97]:
b.characters['i'].agent

{(2, 0, 1): 'am',
 (2, 1, 52): 'mean',
 (2, 2, 63): 'known',
 (2, 3, 101): 'waive',
 (2, 4, 141): 'write',
 (2, 5, 157): 'believe',
 (2, 8, 215): 'saw',
 (7, 0, 11): 'had',
 (7, 6, 195): 'seemed',
 (7, 7, 222): 'are',
 (12, 13, 358): 'was',
 (12, 14, 402): 'have',
 (12, 18, 538): 'had',
 (12, 23, 642): 'reasoned',
 (12, 24, 654): 'was',
 (12, 26, 708): 'presented',
 (12, 27, 746): 'thought',
 (12, 29, 769): 'believe',
 (13, 0, 10): 'had',
 (13, 1, 20): 'was',
 (13, 4, 108): 'spread',
 (17, 0, 9): 'engaged',
 (17, 1, 25): 'sedate',
 (18, 0, 3): 'stated',
 (18, 1, 38): 'threw',
 (18, 2, 49): 'resolved',
 (18, 4, 91): 'placed',
 (18, 6, 193): 'procured',
 (20, 3, 74): 'imagine',
 (20, 4, 113): 'say',
 (22, 0, 6): 'sit',
 (22, 1, 60): 'replied',
 (23, 0, 1): 'sat',
 (23, 2, 43): 'assume',
 (23, 3, 61): 'prefer',
 (24, 0, 6): 'echoed',
 (24, 1, 27): 'mean',
 (24, 2, 30): 'Are',
 (24, 3, 48): 'take',
 (25, 0, 8): 'said',
 (26, 0, 1): 'looked',
 (26, 4, 51): 'been',
 (26, 5, 78): 'have',
 (26